In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
# Define the model
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=20):
        super(MatrixFactorization, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
        # initializing our matrices with a positive number generally will yield better results
        self.user_factors.weight.data.uniform_(0, 0.5)
        self.movie_factors.weight.data.uniform_(0, 0.5)
        
    def forward(self, user, movie):
        return (self.user_factors(user) * self.movie_factors(movie)).sum(1)

In [3]:
# Load the MovieLens dataset
diningHalls = pd.read_csv('./preprocessed-data/alldininghalls.csv')
diningRates = pd.read_csv('./preprocessed-data/dining_ratings.csv')

# Preprocess the data
n_users = diningRates.userId.unique().shape[0]
n_foodItems = diningRates.foodItem.unique().shape[0]

# Convert movieId and userId into unique integers
user_map = {u: i for i, u in enumerate(diningRates.userId.unique())}
diningRates['user_id'] = diningRates['userId'].map(user_map)

dining_map = {m: i for i, m in enumerate(diningRates.foodItem.unique())}
diningRates['food_item'] = diningRates['foodItem'].map(dining_map)

# Create a matrix with users as rows and movies as columns
matrix = torch.zeros((n_users, n_foodItems))
for i, row in diningRates.iterrows():
    matrix[int(row.user_id), int(row.food_item)] = row.rating

In [4]:
model = MatrixFactorization(n_users, n_foodItems)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the model
for i in range(100):
    optimizer.zero_grad()
    user = torch.LongTensor(diningRates.user_id)
    movie = torch.LongTensor(diningRates.food_item)
    rating = torch.FloatTensor(diningRates.rating)
    predictions = model(user, movie)
    loss = criterion(predictions, rating)
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(loss)


tensor(6.1235, grad_fn=<MseLossBackward0>)
tensor(6.1091, grad_fn=<MseLossBackward0>)
tensor(6.0948, grad_fn=<MseLossBackward0>)
tensor(6.0805, grad_fn=<MseLossBackward0>)
tensor(6.0663, grad_fn=<MseLossBackward0>)
tensor(6.0522, grad_fn=<MseLossBackward0>)
tensor(6.0381, grad_fn=<MseLossBackward0>)
tensor(6.0241, grad_fn=<MseLossBackward0>)
tensor(6.0102, grad_fn=<MseLossBackward0>)
tensor(5.9963, grad_fn=<MseLossBackward0>)


In [5]:
# Make recommendations for a given user
def recommend_movies(model, user_id, num_recommendations):
    with torch.no_grad():
        user = torch.LongTensor([user_map[user_id]])
        movies = torch.arange(n_foodItems)
        ratings = model(user, movies).detach().numpy()
    movie_ids = ratings.argsort()[-num_recommendations:][::-1]
    recommended_movies = [movies[i] for i in movie_ids]
    return recommended_movies

In [9]:
# Get recommendations for a user with user_id 1
def getRecs(model, user_id, num_recs):
    recommended_movies = recommend_movies(model, user_id, num_recs)

    # Convert tensors to Int
    val = []
    for i in range(num_recs):
        val.append(int(recommended_movies[i]))

    for id in val:
        row = diningHalls.loc[diningHalls['foodId'] == id]
        # print(type(row))
        movie = row.values.tolist()
        if len(movie) == 0:
            continue
        print(movie[0][1])
        
def getUserInfo(user):
    row = diningRates.loc[diningRates['userId'] == user]
    x = row.loc[diningRates['rating'] >= 5.0]
    foodIDS = x['foodItem'].values

    for id in foodIDS:
        movie = diningHalls.loc[diningHalls['foodId'] == id]
        a = movie.values.tolist()
        try:
            print(a[0][1])
        except:
            pass


In [11]:
userID = 34
print("Recommendations for User {}: ".format(userID))
print()
getRecs(model, userID, 5)
print()
print("----------------------------------------------------------------------------------------")
print()
print("Food Items that User {} has rated over 5.0:".format(userID)) 
print()
getUserInfo(userID)

Recommendations for User 34: 

Honey Glazed Carrots
Sticky Rice
Ziti and Beef Cheesy Bake
Grilled Cheese

----------------------------------------------------------------------------------------

Food Items that User 34 has rated over 5.0:

Oatmeal Cereal
French Toast
Bacon
Grilled Chorizo Sausage
Broccoli Saute
Roasted Asparagus
Sticky Rice
Wood Grilled Cheese Pizza
Roasted Cauliflower
Grilled Cheese
Grilled Chorizo Sausage


IndexError: list index out of range